# [Модуль "Pythob для аналитиков данных"](https://ru.hexlet.io/programs/python-for-data-analysts)

## [Проект: Дашборд конверсий](https://ru.hexlet.io/programs/python-for-data-analysts/projects/100)


In [2]:
import pandas as pd
import re
import requests

#### Данные (образец):

[Визиты](https://drive.google.com/file/d/1QosQQ4RRNR9rkL4t7sB707h2Uy0XfYJe/view?usp=drive_link) - тысяча записей с визитами

[Регистрации](https://drive.google.com/file/d/1AeQz0kaSgz0lxYSDtuNm36muhy5fRCzZ/view?usp=drive_link) - тысяча записей о первых регистрациях


## Скачивание данных

In [6]:
API_URL = 'https://data-charts-api.hexlet.app'
DATE_BEGIN = '2023-03-01'
DATE_END = '2023-09-01'

In [8]:
def download_from_api(url, endpoint, path_to_save, start_date=None, end_date=None):
    download_link = f'{url}/{endpoint}'
    if start_date or end_date:
        download_link += "?"
        if start_date:
            download_link += f'begin={start_date}'
        if end_date:
            download_link += f'&end={end_date}' if start_date else f'end={end_date}'
            
    print(f'Download from the link {download_link}...')
    response = requests.get(download_link)
    if response.ok:
        with open(path_to_save, 'wb') as file:
            file.write(response.content)
        print(f"File successfully saved as '{path_to_save}'")
    else:
        print(f"Error downloading file: status code {response.status_code}")

In [35]:
visits_path = './data/visits.json'
registrations_path = './data/registrations.json'

#### Скачивание данных **visits**

In [12]:
download_from_api(url=API_URL,
         endpoint='visits',
         path_to_save=visits_path,
         start_date=DATE_BEGIN,
         end_date=DATE_END)

Download from the link https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01...
File successfully saved as './data/visits.json'


#### Скачивание данных **registrations**

In [43]:
download_from_api(url=API_URL,
         endpoint='registrations',
         path_to_save=registrations_path,
         start_date=DATE_BEGIN,
         end_date=DATE_END)

Download from the link https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01...
File successfully saved as './data/regsistrations.json'


## Предварительная подготовка датафреймов

### visits

In [22]:
visits = pd.read_json(visits_path)
visits.head(3)

,visit_id,platform,user_agent,datetime
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01 10:36:22
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,2023-03-01 06:25:00
2,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01 10:26:13


In [24]:
# Преобразование datetime к date, что более уместно для наших задач
visits['date'] = pd.to_datetime(visits.pop('datetime')).dt.date
visits.head(3)

,visit_id,platform,user_agent,date
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,2023-03-01
2,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01


In [26]:
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263459 entries, 0 to 263458
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   visit_id    263459 non-null  object
 1   platform    263459 non-null  object
 2   user_agent  263459 non-null  object
 3   date        263459 non-null  object
dtypes: object(4)
memory usage: 8.0+ MB


In [28]:
visits.describe()

,visit_id,platform,user_agent,date
count,263459,263459,263459,263459
unique,146085,4,32,184
top,64167edd-323a-4ab0-be9b-acd237a1ac30,web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,2023-03-08
freq,4,236301,13623,2624


In [30]:
visits.head(3)

,visit_id,platform,user_agent,date
0,1de9ea66-70d3-4a1f-8735-df5ef7697fb9,web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,2023-03-01
1,f149f542-e935-4870-9734-6b4501eaf614,web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,2023-03-01
2,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1,web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,2023-03-01


### registrations

In [37]:
registrations = pd.read_json(registrations_path)
registrations.head(3)

,datetime,email,platform,registration_type,user_id
0,2023-03-01 07:40:13+00:00,ebyrd@example.org,web,google,2e0f6bb8-b029-4f45-a786-2b53990d37f1
1,2023-03-01 13:14:00+00:00,knightgerald@example.org,web,email,f007f97c-9d8b-48b5-af08-119bb8f6d9b6
2,2023-03-01 03:05:50+00:00,cherylthompson@example.com,web,apple,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15


In [39]:
# Преобразование datetime к date, что более уместно для наших задач
registrations['date'] = pd.to_datetime(registrations.pop('datetime')).dt.date
registrations.head(3)

,email,platform,registration_type,user_id,date
0,ebyrd@example.org,web,google,2e0f6bb8-b029-4f45-a786-2b53990d37f1,2023-03-01
1,knightgerald@example.org,web,email,f007f97c-9d8b-48b5-af08-119bb8f6d9b6,2023-03-01
2,cherylthompson@example.com,web,apple,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15,2023-03-01


In [41]:
registrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21836 entries, 0 to 21835
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   email              21836 non-null  object
 1   platform           21836 non-null  object
 2   registration_type  21836 non-null  object
 3   user_id            21836 non-null  object
 4   date               21836 non-null  object
dtypes: object(5)
memory usage: 853.1+ KB


In [43]:
registrations.describe()

,email,platform,registration_type,user_id,date
count,21836,21836,21836,21836,21836
unique,20868,3,4,21836,184
top,ujones@example.com,android,email,2e0f6bb8-b029-4f45-a786-2b53990d37f1,2023-03-06
freq,6,10582,8996,1,230


In [45]:
registrations.head(3)

,email,platform,registration_type,user_id,date
0,ebyrd@example.org,web,google,2e0f6bb8-b029-4f45-a786-2b53990d37f1,2023-03-01
1,knightgerald@example.org,web,email,f007f97c-9d8b-48b5-af08-119bb8f6d9b6,2023-03-01
2,cherylthompson@example.com,web,apple,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15,2023-03-01


## Расчет метрик

* Сгруппируйте данные визитов по датам и платформам
* Сгруппируйте также данные регистраций по датам и платформам
* Объедините датайфреймы, сделайте итоговый датафрейм с расчетом конверсии
* Сохраните датафрейм в формате JSON conversion.json
* Поля датафрейма:
  - date_group - сагрегированный столбец дат
  - platform - платформа (android,web,ios)
  - visits - визиты за период date_group
  - registrations - регистрации за период date_group
  - conversion - конверсия по платформе

#### Группировка данных визитов по датам и платформам

##### Визиты ботов не должны влиять на расчет конверсии

In [89]:
visits_withot_bots = visits[~visits.user_agent.str.contains('bot')]

In [99]:
visits_grouped = visits_withot_bots.groupby(['date','platform']).agg(visits=('visit_id', 'size')).reset_index()
visits_grouped.rename(columns={'date': 'date_group'}, inplace=True)
visits_grouped

,date_group,platform,visits
0,2023-03-01,android,75
1,2023-03-01,ios,22
2,2023-03-01,web,844
3,2023-03-02,android,67
4,2023-03-02,ios,31
...,...,...,...
547,2023-08-30,ios,66
548,2023-08-30,web,1357
549,2023-08-31,android,57
550,2023-08-31,ios,50


In [103]:
registrations_grouped = registrations.groupby(['date','platform']).agg(visits=('visit_id', 'size')).reset_index()
# registrations_grouped.rename(columns={'date': 'date_group'}, inplace=True)
registrations_grouped

KeyError: "Column(s) ['visit_id'] do not exist"